<a href="https://colab.research.google.com/github/ghluque/Analisis_BTC/blob/master/BTC_TX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Leer transacciones de la blockchain

In [0]:
import pandas as pd
import numpy as np

import requests #U: leer de una url 
import json #U: leer json, PORQUE pandas solo lee json con forma de data frame!


In [0]:
block_url="https://blockchain.info/rawblock/0000000000000000000cfe48631b9dc8083201f7bda4deb5bc081e344d1a5913"

In [0]:
block= json.loads(requests.get(block_url).text) #A: block es un diccionario de diccionarios y arrays

In [0]:
block['tx'][2:4] #A: las transacciones 2 a 4, que propiedades tienen?

[{'balance': 0,
  'block_height': 619002,
  'block_index': 0,
  'double_spend': False,
  'fee': 100000,
  'hash': '12565431a8f14cc4e65767b76914969474652fa0b69b02e8fcf177f977e9e5d7',
  'inputs': [{'index': 0,
    'prev_out': {'addr': '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX',
     'n': 2,
     'script': '76a91443849383122ebb8a28268a89700c9f723663b5b888ac',
     'spending_outpoints': [{'n': 0, 'tx_index': 0}],
     'spent': True,
     'tx_index': 0,
     'type': 0,
     'value': 253537196},
    'script': '483045022100d981306e2cb0ef59167c408d3dbd81e81af36e9e105ccbf0d110b3d8af39c6f50220246b4e8e7e36be43148778ea6fb8658728535fcdc31503fb821d083f71654d9d0141047146f0e0fcb3139947cf0beb870fe251930ca10d4545793d31033e801b5219abf56c11a3cf3406ca590e4c14b0dab749d20862b3adc4709153c280c2a78be10c',
    'sequence': 4294967295,
    'witness': ''}],
  'lock_time': 0,
  'out': [{'addr': '325kjwqJKbcCnJ2LhjqmmeXaPhYiTZ1EFN',
    'n': 0,
    'script': 'a914044ce4aa9472b406557ae47b52b98d098dc29d7a87',
    'spending_o

In [0]:
IN_CNT_MAX=5 #U: maximo de inputs por tx
OUT_CNT_MAX=5 #U: maximo de outputs por tx
Tx_keys= ['hash','time']
Tx_normalized_cols= Tx_keys + ['i'+str(n)+'v' for n in range(0,IN_CNT_MAX)] + ['o'+str(n)+'v' for n in range(0,OUT_CNT_MAX)]
def pad_list(l,desiredLen):
  """Agrega 0 a la lista hasta alcanzar desiredLen"""
  return l + [0] * (desiredLen-len(l)) if len(l)<desiredLen else l[0:desiredLen]

def tx_in_normalized(tx):
  return [(x['prev_out']['value'] if 'prev_out' in x else 0) for x in tx['inputs']]

def tx_out_normalized(tx):
  return [x['value'] for x in tx['out']]

def tx_normalized(tx):
  """Devuelve una tx de la blockchain como una lista con hash, value para 5 inputs, value 5 outputs"""
  return [tx[k] for k in Tx_keys] + pad_list(tx_in_normalized(tx),IN_CNT_MAX) + pad_list(tx_out_normalized(tx),OUT_CNT_MAX)

In [0]:
tx= pd.DataFrame( [ tx_normalized(tx) for tx in block['tx'] ], columns= Tx_normalized_cols)
#A: normalizamos nuestro bloque para que tenga forma de tabla y creamos un dataframe
tx.time= pd.to_datetime(tx.time, unit='s') #A: convertimos el time del bloque en un datetime

tx.head()


,hash,time,i0v,i1v,i2v,i3v,i4v,o0v,o1v,o2v,o3v,o4v
0,bfcc1c233827fbf147b9744b58844465abf61232c02fb2...,2020-02-26 00:10:06,0,0,0,0,0,1276409435,0,0,0,0
1,8325d3decbb805e09f042fc946bc61076be3578af331b1...,2020-02-25 23:55:16,50000000000,0,0,0,0,5100000000,44899916000,0,0,0
2,12565431a8f14cc4e65767b76914969474652fa0b69b02...,2020-02-25 23:46:22,253537196,0,0,0,0,1180000,7125764,245131432,0,0
3,0f24da007c306726713906933e0e171800290cbf5e3047...,2020-02-25 23:55:45,245131432,0,0,0,0,7875000,9994323,227162109,0,0
4,9491757c01ab9c01eb03d6db97e4feba4c4675bf6dd803...,2020-02-26 00:08:50,777883277,0,0,0,0,10700520,767132657,0,0,0


In [0]:
#OjO! de los outputs uno suele ser el fee, y otro suele ser al mismo dueño de la billetera que paga ... 
# porque si en una tx anterior compre 3btc y te quiero pagar 1btc,
# mando 1btc a tu billetera, y 2 a la mia de nuevo (el vuelto)
# por ej.
print("Direccion anterior {}\nEl vuelto va a {}\n",[ block['tx'][2]['inputs'][0]['prev_out']['addr'], block['tx'][2]['out'][2]['addr'] ])


Direccion anterior {}
El vuelto va a {}
 ['17A16QmavnUfCW11DAApiJxp7ARnxN5pGX', '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX']
